In [1]:
# Update:
# added more parameter controls.
# testing on github

In [2]:
import import_ipynb
from simulation_structure_V1 import *
import ipywidgets as widgets

import io
import pandas as pd

importing Jupyter notebook from Node_xiaotian_V10.ipynb


In [3]:
from ipywidgets import interact, HBox, VBox,interactive_output
from IPython.display import HTML
import numpy as np
import io
import base64

In [4]:
# arrange plot layout
# source:https://stackoverflow.com/questions/21754976/ipython-notebook-arrange-plots-horizontally

class FlowLayout(object):
    ''' A class / object to display plots in a horizontal / flow layout below a cell '''
    def __init__(self):
        # string buffer for the HTML: initially some CSS; images to be appended
        self.sHtml =  """
        <style>
        .floating-box {
        display: inline-block;
        margin: 10px;
        border: 3px solid #888888;  
        }
        </style>
        """

    def add_plot(self, oAxes):
        ''' Saves a PNG representation of a Matplotlib Axes object '''
        Bio=io.BytesIO() # bytes buffer for the plot
        fig = oAxes.get_figure()
        fig.canvas.print_png(Bio) # make a png of the plot in the buffer

        # encode the bytes as string using base 64 
        sB64Img = base64.b64encode(Bio.getvalue()).decode()
        self.sHtml+= (
            '<div class="floating-box">'+ 
            '<img src="data:image/png;base64,{}\n">'.format(sB64Img)+
            '</div>')

    def PassHtmlToCell(self):
        ''' Final step - display the accumulated HTML '''
        display(HTML(self.sHtml))
        

## Set inputs
##### Units: distance:km;  time:min

##### policy_para_1: if the number of EVs exceeds capacity*policy_para_1, then relocation happens.

In [5]:
def set_inputs(station_loca,demand,zoom,max_soc,horizon_len,time_interval_len,consumption_rate,
              charging_rate,rtr):
    global md,nw
    time_interval_num=math.ceil(horizon_len/time_interval_len)

    try:
        # read station locations
        data =pd.read_csv(io.BytesIO(station_loca[[*station_loca][0]]["content"]))
        station_location=[]
        station_capacity=[]
        station_initial_ev_num=[]
        for i in range(len(data)):
            station_location.append([data['Latitude'][i],data['Longitude'][i]]) 
            station_capacity.append(data['capacity'][i])
            station_initial_ev_num.append(data['starting_EV_amount'][i])
        ave_occupancy=sum(station_initial_ev_num)/sum(station_capacity)
    except:
        print('\033[91m'+'upload a .csv file for station locations. example:'+'\x1b[0m')
        
    try:
        # read demand
        d_data=pd.read_csv(io.BytesIO(demand[[*demand][0]]["content"]))
        demand_dic={}
        for i in range(len(d_data)):
            demand_dic[d_data['origin'][i],d_data['destination'][i],d_data['period'][i]]=d_data['amount'][i]
        
        md=MapDisplay(station_location,zoom)
        
        # obtain distance matrix
        distance_2={}
        for i in range(md.station_num):
            for j in range(md.station_num):
                distance_2[i,j]=math.ceil(md.distance_matrix[i][j]/time_interval_num)
        
        targeting_occupancy=ave_occupancy
        nw=Network(station_location,distance_2,max_soc,time_interval_num,consumption_rate,charging_rate,station_capacity,500,demand_dic,[rtr,targeting_occupancy])
        nw.deploy_vehicle(station_initial_ev_num)
        nw.run()
        md.take_sim_result(nw.sim_result,nw.max_soc)
        # add click event
        md.show_station_status()
    except:
        print('\033[91m'+'upload a .csv file for demands. example:'+'\x1b[0m')


In [6]:
station_loca_file=widgets.FileUpload(
                description='location file',
                accept='.csv',
                multiple=False  # True to accept multiple files upload else False
                            )
demand_file=widgets.FileUpload(
                description='demand file',
                accept='.csv',  
                multiple=False 
                            )
zoom_level=widgets.IntSlider(min=1,max=16,step=1,value=13,description='zoom')
max_soc=widgets.IntSlider(min=1,max=15,step=1,value=7,description='max state of charge',
                         style = {'description_width': 'initial'})
horizon_len=widgets.IntSlider(min=1,max=24,step=1,value=10,description='operation_horizon_length(hour)',
                          style = {'description_width': 'initial'})
time_interval_len=widgets.FloatSlider(min=0.1,max=24,step=0.1,value=1.0,description='time_interval_length(hour)',
                          style = {'description_width': 'initial'})
consumption_rate=widgets.IntSlider(min=1,max=5,step=1,value=2,description='battery comsumption rate',
                          style = {'description_width': 'initial'})
charging_rate=widgets.IntSlider(min=1,max=5,step=1,value=1,description='battery charging rate',
                          style = {'description_width': 'initial'})
relocation_trigger_ratio=widgets.FloatSlider(min=0.1,max=1.0,step=0.1,value=0.8,description='policy_para_1',
                          style = {'description_width': 'initial'})
box_row_1=HBox([station_loca_file,demand_file])
box_row_2=HBox([horizon_len,zoom_level])
box_row_3=HBox([time_interval_len,max_soc])
box_row_4=HBox([consumption_rate,charging_rate])
box_row_5=HBox([relocation_trigger_ratio])
ui_map=VBox([box_row_1,box_row_2,box_row_3,box_row_4,box_row_5])
out_map=interactive_output(set_inputs,{'station_loca':station_loca_file,'demand':demand_file,'zoom':zoom_level,
                                       'max_soc':max_soc,'horizon_len':horizon_len,'time_interval_len':time_interval_len,
                                      'consumption_rate':consumption_rate,'charging_rate':charging_rate,
                                      'rtr':relocation_trigger_ratio})
display(ui_map,out_map)


Output()

#### Click a station to show index; Then specify time and station to show simulation results

In [7]:
# show node information
def node_info(t,station):
    oPlot = FlowLayout() # create an empty FlowLayout
    try:
#         nw.show_route(t,1,0)
#         nw.show_node(t,nw.stations[1])
#         nw.show_map(t)
#         nw.show_node_histogram(t,nw.stations[1])
        md.show_station_status(t) # this is needed to update popup info
        ax_1=nw.show_node(t,nw.stations[station])
        
        plt.savefig('show_node.png')
        
        plt.close() # close the figure directly plot by calling nw methods
        ax_2=nw.show_node_histogram(t,nw.stations[station])
        
        plt.savefig('show_histogram.png')
        
        plt.close()
        oPlot.add_plot(ax_1)
        oPlot.add_plot(ax_2)
        oPlot.PassHtmlToCell()
    except:
        print('\033[91m'+'error in station index'+'\x1b[0m')

In [8]:
# md.m.save('my_map.html', title='My Map')

In [9]:
t_slider = widgets.IntSlider(min=0,max=9,step=1,value=0,description='t')
station_IntText=widgets.BoundedIntText(
    value=1e10,
    min=0,
    max=1e10,
    step=1,
    description='Station',
    disabled=False
)
ui_node=HBox([t_slider,station_IntText])
out_node=interactive_output(node_info,{'t':t_slider,'station':station_IntText})
display(ui_node,out_node)

Output()

#### Route information between two stations

In [10]:
def route_info(t,origin,destination):
#     ax_1=nw.show_route(t,origin,destination)
#     plt.close()
# #     fig=ax_1.get_figure()
    try:
        if origin<=md.station_num-1 and destination<=md.station_num-1:
            nw.show_route(t,origin,destination)
            
            plt.xlabel('from station 4 to station 1')
            plt.savefig('show_route.png')
            
        else:
            print('\033[91m'+'error in station index'+'\x1b[0m')

    except:
        print('\033[91m'+'error in station index'+'\x1b[0m')

In [11]:
origin_IntText=widgets.BoundedIntText(
    value=1e10,
    min=0,
    max=1e10,
    step=1,
    description='Origin station',
    style = {'description_width': 'initial'},
    disabled=False
)
destination_IntText=widgets.BoundedIntText(
    value=1e10,
    min=0,
    max=1e10,
    step=1,
    description='Destination station',
    style = {'description_width': 'initial'},
    disabled=False
)
out_route=interactive_output(route_info,{'t':t_slider,'origin':origin_IntText,'destination':destination_IntText})
# use the same time slider, but not to display it. 
ui_route=HBox([origin_IntText,destination_IntText])
display(ui_route,out_route)

Output()